In [24]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets

from models import PNet
from dataset_pnet import FaceDataset
from train_utils_pnet import *

from config import args

In [25]:
normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                 std=[0.5, 0.5, 0.5])

train_dataset = FaceDataset(
    args.data,
    transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))


train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=args.batch_size, shuffle=True,
    num_workers=args.workers, pin_memory=True)

In [26]:
model = PNet()

if args.gpu is not None:
    model = model.cuda()
    #model = model
    
optimizer = torch.optim.SGD(model.parameters(), args.lr)
cudnn.benchmark = True
#cudnn.benchmark = False

criterion = nn.CrossEntropyLoss().cuda(args.gpu)
criterion_reg = nn.MSELoss().cuda(args.gpu)
#criterion = nn.CrossEntropyLoss()

In [27]:
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, labels, reg) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if args.gpu is not None:
            input = input.cuda(args.gpu)
            reg = reg.cuda(args.gpu)
            labels = labels.cuda(args.gpu)

        # compute output
        logits, prob, reg_2 = model(input)
        
        lab_neg=(labels==0).nonzero()
        lab_pos=(labels==1).nonzero()
        lab_part=(labels==-1).nonzero()
        
        label_clc=torch.cat((lab_neg, lab_pos), 0)
        label_reg=torch.cat((lab_part, lab_pos), 0)
        
        #print(label_clc.view(-1))
        #print(label_reg.view(-1))
        #print(logits.view(-1, 2).size())
        #print(logits.view(-1, 2)[0].size())
        #lab_cls=
        #print((labels==0).nonzero().size())
        #print((labels==-1).nonzero().size())
        reg=reg.type(torch.cuda.FloatTensor)
        #print(labels.size())
        #(a==4).nonzero()[0].squeeze()
        #len_vec=range(len(lab_neg)+len(lab_pos))
        #x = torch.zeros(len(lab_neg)+len(lab_pos), 2).cuda(args.gpu)
        #y = torch.zeros(len(lab_neg)+len(lab_pos), 2).cuda(args.gpu)
        #index=torch.tensor(len_vec).cuda(args.gpu)
        #for i in range(len(lab_neg)):
            #res=lab_neg[i]
            #print(res)
            #print(logits.view(-1, 2)[res])
            #x.index_add_(0, index, logits.view(-1, 2)[res])
            #print(x)
        
        clc=torch.index_select(logits.view(-1, 2), 0, label_clc.view(-1))
        l_clc=torch.index_select(labels, 0, label_clc.view(-1))
        #print(clc)
        #print(l_clc)
        
        
        regr=torch.index_select(reg_2.view(-1, 4), 0, label_reg.view(-1))
        l_regr=torch.index_select(reg, 0, label_reg.view(-1))
        #print(regr)
        #print(l_regr)
        #loss = criterion(logits.view(-1, 2), labels)+criterion_reg(reg_2.view(-1, 4),reg)
        loss = criterion(clc, l_clc)+criterion_reg(regr,l_regr)
        #print(loss)
        
        
        # measure accuracy and record loss
        prec1 = accuracy(logits.view(-1, 2), labels, topk=(1,))
        losses.update(loss.item(), input.shape[0])
        top1.update(prec1[0], input.shape[0])

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        if i % args.print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1_val:.3f} ({top1_avg:.3f})'.format(
                   epoch, i, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1_val=top1.val.item(), top1_avg=top1.avg.item()))
    return top1.avg.item()

In [ ]:
best_prec = 0

for epoch in range(args.epochs):
    adjust_learning_rate(optimizer, epoch, args.lr)

    # train for one epoch
    prec = train(train_loader, model, criterion, optimizer, epoch)
    
    save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'best_prec': best_prec,
        }, prec > best_prec)
    
    best_prec = max(prec, best_prec)

Epoch: [0][0/3899]	Time 132.868 (132.868)	Data 132.852 (132.852)	Loss 0.7525 (0.7525)	Prec@1 19.141 (19.141)


In [12]:
x = torch.Tensor(5,6)
#print(x)
#print((x==0).nonzero())
print(x.view(-1))

tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.])


In [7]:
# TODO:
# 1) BBOX regression with "part" images (label == -1)
# 2) Validation set
# 3) Tune the parameters
# 4) Augmentations